In [3]:
import posix_ipc
import mmap
import numpy as np
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq
import glob

In [2]:
shm = posix_ipc.SharedMemory(name='/shm_20231226_order')

In [3]:
shm_mmap = mmap.mmap(shm.fd, shm.size, access=mmap.ACCESS_READ)

In [4]:
shm_arr = np.ndarray((118490170, 7), dtype='object', buffer=shm_mmap)

In [ ]:
print(shm_arr)

: 

In [8]:
df = pd.DataFrame(shm_arr)

: 

In [11]:
shm = posix_ipc.SharedMemory(name='/shm_20231229_order')
# Map the shared memory into the process's address space
shm_mmap = mmap.mmap(shm.fd, shm.size, access=mmap.ACCESS_READ)
# Create a NumPy array from the mapped memory
shm_arr = np.ndarray((138161989, 7), dtype='float64', buffer=shm_mmap)
df = pd.DataFrame(shm_arr)
df

,0,1,2,3,4,5,6
0,1.703813e+09,248.0,23.30,3600.0,1.0,4.0,601699.0
1,1.703813e+09,818.0,22.50,400.0,1.0,4.0,601699.0
2,1.703813e+09,1103.0,23.84,100.0,1.0,4.0,601699.0
3,1.703813e+09,2716.0,21.97,500.0,1.0,4.0,601699.0
4,1.703813e+09,3324.0,22.29,3000.0,1.0,4.0,601699.0
...,...,...,...,...,...,...,...
138161984,1.703833e+09,33482493.0,6.99,2400.0,0.0,2.0,2451.0
138161985,1.703833e+09,33482521.0,7.00,1200.0,0.0,2.0,2451.0
138161986,1.703833e+09,33482902.0,6.99,7800.0,0.0,2.0,2451.0
138161987,1.703833e+09,33482956.0,7.00,1500.0,0.0,2.0,2451.0


In [7]:
data_path = '/home/haolinl/converted_parquet/test.parquet'
# Load data as a Pandas DataFrame
df = pd.read_parquet(data_path)

In [8]:
# Convert to NumPy array for sharing
array = df.to_numpy()
# Create shared memory


In [13]:
shm_name = f"/shm_20231226_order"  # Shared memory name must start with a slash
shm = posix_ipc.SharedMemory(
    name=shm_name,
    flags= posix_ipc.O_CREAT | posix_ipc.O_EXCL,
    mode=0o600,
    size=array.nbytes
)
shm_mmap = mmap.mmap(shm.fd, shm.size, access=mmap.ACCESS_WRITE)
# Create a NumPy array from the mapped memory
shm_arr = np.ndarray(array.shape, dtype=array.dtype, buffer=shm_mmap)
shm_arr[:] = array[:]

In [12]:
shm = posix_ipc.SharedMemory(name='/shm_20231226_order')
# Map the shared memory into the process's address space
shm_mmap = mmap.mmap(shm.fd, shm.size, access=mmap.ACCESS_READ)
# Close the mmap object (shared memory remains open for other processes)
shm_mmap.close()
# Close the file descriptor (shared memory remains open for other processes)
shm.close_fd()
shm.unlink()  # Remove the shared memory segment

In [20]:
data_path = '/home/Level2/2023/20231226/sh600030.h5'
# Load data as a Pandas DataFrame
df = pd.read_hdf(data_path,'order')
# Convert to NumPy array for sharing
array = df.to_numpy()
# Create shared memory
shm_name = f"/shm_20231226_order"  # Shared memory name must start with a slash
try:
    # Create a new shared memory segment
    shm = posix_ipc.SharedMemory(
        name=shm_name,
        flags= posix_ipc.O_CREAT | posix_ipc.O_EXCL,
        mode=0o600,
        size=array.nbytes
    )
except posix_ipc.ExistentialError:
    # If the segment already exists, open it
    shm = posix_ipc.SharedMemory(name=shm_name)
    # Resize the segment if necessary
    if shm.size < array.nbytes:
        os.ftruncate(shm.fd, array.nbytes)
# Map the shared memory into the process's address space
shm_mmap = mmap.mmap(shm.fd, shm.size, access=mmap.ACCESS_WRITE)
# Create a NumPy array from the mapped memory
shm_arr = np.ndarray(array.shape, dtype=array.dtype, buffer=shm_mmap)
shm_arr[:] = array[:]

# Close the mmap object (shared memory remains open for other processes)
shm_mmap.close()
# Close the file descriptor (shared memory remains open for other processes)
shm.close_fd()
print(shm_name,shm_arr.shape,shm_arr.dtype)

/shm_20231226_order (96419, 6) float64


In [15]:


# Store shared memory name, shape, and dtype in the cache
print(f"Shared memory name: {shm_name}; shape: {array.shape}; dtype: {array.dtype}")

Shared memory name: /shm_20231226_order; shape: (96419, 6); dtype: float64


In [21]:
def write_to_parquet(df, path):
    table = pa.Table.from_pandas(df)
    pq.write_table(
        table,
        path,
        compression='snappy',
        row_group_size=100000  # 设置 Row Group 的大小
    )

In [4]:
df = pd.read_hdf('/home/Level2/2023/20231226/sh600030.h5','order')
df['stock_code'] = 'sh600030'
write_to_parquet(df, '/home/haolinl/converted_parquet/test_withcode.parquet')

In [45]:
fp = '/home/haolinl/converted_parquet/20231226_trade.parquet'
df = pd.read_parquet(fp)
df.dtypes
# df['stock_code'] = df['stock_code'].str[2:].astype('float64')
# write_to_parquet(df,fp)

TradeTime      float64
sysid          float64
TradeCode      float64
BSFlag         float64
TradePrice     float64
TradeVolume    float64
SellOrderID    float64
BuyOrderID     float64
stock_code     float64
dtype: object

In [ ]:
date_dirs = os.listdir('/home/Level2/2023')
to_date_time = lambda x: pd.to_datetime(x, format='%Y%m%d', errors='coerce')
date_dirs = [to_date_time(date) for date in date_dirs]
date_dirs = [date for date in date_dirs if date>=pd.to_datetime('20231101', format='%Y%m%d') and date<=pd.to_datetime('20231131', format='%Y%m%d')]
date_dirs

[Timestamp('2023-12-26 00:00:00'),
 Timestamp('2023-08-23 00:00:00'),
 Timestamp('2023-07-13 00:00:00'),
 Timestamp('2023-12-22 00:00:00'),
 Timestamp('2023-05-31 00:00:00'),
 Timestamp('2023-05-29 00:00:00'),
 Timestamp('2023-04-21 00:00:00'),
 Timestamp('2023-08-29 00:00:00'),
 Timestamp('2023-04-04 00:00:00'),
 Timestamp('2023-11-23 00:00:00'),
 Timestamp('2023-10-12 00:00:00'),
 Timestamp('2023-09-26 00:00:00'),
 Timestamp('2023-07-31 00:00:00'),
 Timestamp('2023-04-13 00:00:00'),
 Timestamp('2023-11-20 00:00:00'),
 Timestamp('2023-06-19 00:00:00'),
 Timestamp('2023-08-01 00:00:00'),
 Timestamp('2023-08-15 00:00:00'),
 Timestamp('2023-10-09 00:00:00'),
 Timestamp('2023-01-12 00:00:00'),
 Timestamp('2023-07-11 00:00:00'),
 Timestamp('2023-03-17 00:00:00'),
 Timestamp('2023-03-03 00:00:00'),
 Timestamp('2023-07-19 00:00:00'),
 Timestamp('2023-11-21 00:00:00'),
 Timestamp('2023-09-06 00:00:00'),
 Timestamp('2023-11-22 00:00:00'),
 Timestamp('2023-04-20 00:00:00'),
 Timestamp('2023-09-